In [89]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import nltk

---

In [90]:
user = pd.read_csv('user_df.csv')
user.Corpus = user.Corpus.astype('str')
print(user.isna().sum())
user.info()

Unnamed: 0     0
ApplicantID    0
Corpus         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15959 entries, 0 to 15958
Data columns (total 3 columns):
Unnamed: 0     15959 non-null int64
ApplicantID    15959 non-null int64
Corpus         15959 non-null object
dtypes: int64(2), object(1)
memory usage: 374.1+ KB


In [91]:
job = pd.read_csv('jobs_df.csv')
job.Corpus = job.Corpus.astype('str')
print(job.isna().sum())
job.info()

Unnamed: 0    0
JobID         0
Corpus        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84090 entries, 0 to 84089
Data columns (total 3 columns):
Unnamed: 0    84090 non-null int64
JobID         84090 non-null int64
Corpus        84090 non-null object
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


---

In [92]:
def select_applicant(user_id, dataframe):
    '''
    Selects the given users first application
    returns: selected applicants row
    input: user id
    '''
    index = np.where(dataframe.ApplicantID == user_id)[0][0]
    return dataframe.iloc[index]

In [93]:
applicant = select_applicant(10000, user)
print(applicant)

Unnamed: 0                                                   0
ApplicantID                                              10000
Corpus         cashiers   valets needed  wallypark  newark    
Name: 0, dtype: object


---
### TF-IDF
( Term Frequency - Inverse Document Frequency )

In [94]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [95]:
vectorizer = TfidfVectorizer()

tf_idf_job = vectorizer.fit_transform(job.Corpus)
tf_idf_job

<84090x43007 sparse matrix of type '<class 'numpy.float64'>'
	with 8039613 stored elements in Compressed Sparse Row format>

In [96]:
tf_idf_user = vectorizer.transform(user.Corpus)
tf_idf_user

<15959x43007 sparse matrix of type '<class 'numpy.float64'>'
	with 81239 stored elements in Compressed Sparse Row format>

In [98]:
output = map(lambda x: cosine_similarity(tf_idf_user, x), tf_idf_job)
output_list = list(output)